In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


'''def train_network():
    """ Train a Neural Network to generate music """
    notes, note_num = get_notes()

    # get amount of pitch names
#     a = set(notes)
#     print("a:")
#     print(a)
#     n_vocab = len(a)
#     print(n_vocab)

    network_input, network_output = prepare_sequences(notes, note_num)

    model = create_network(network_input, note_num)
    
    train(model, network_input, network_output)'''

    
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """

    notes = numpy.zeros((1,88))

    for file in glob.glob("test/*.mid"):
        midi = converter.parse(file)


        print("Parsing %s" % file)

        notes_to_parse = None

        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)

            notes_to_parse = s2.parts[0].recurse()

        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        
        notes_matrix = numpy.zeros((note_num, 88))
    
        count = 0
        
        for element in notes_to_parse:
            temp_matrix =  numpy.zeros(88)
            if isinstance(element, note.Note):
#                 notes.append(str(element.pitch))
                temp_matrix[element.pitch.midi-21] = 1
                notes_matrix[count] = temp_matrix
                count += 1
            elif isinstance(element, chord.Chord):
#                 print(element)
                for p in element.pitches:
                    temp_matrix[p.midi-21] = 1
                notes_matrix[count] = temp_matrix
                count += 1
            
        notes = numpy.concatenate((notes, notes_matrix), axis=0)
#         print(notes_matrix)
#         print(notes)
            
#                     print(p.midi)
#                 for n in element.normalOrder:
#                     print(n)
#                 notes.append('.'.join(str(n) for n in element.normalOrder))
#         print(notes_matrix)
    notes = numpy.delete(notes, 0, axis = 0)

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes_matrix, filepath)
    
    return notes

def prepare_sequences(notes, note_num):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
#     pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
#     note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
#     print(note_to_int)

    network_input = numpy.zeros(((note_num - sequence_length, sequence_length, 88)))
    network_output = numpy.zeros((note_num - sequence_length, 88))
#     network_input = []
#     network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, note_num - sequence_length, 1):
        
#         sequence_in = notes[i:i + sequence_length]
#         sequence_out = notes[i + sequence_length]
#         network_input.append([note_to_int[char] for char in sequence_in])
#         network_output.append(note_to_int[sequence_out])
#         for j in range(0, sequence_length, 1):
#             print(network_input[i, j])
#             print(notes[i + j,])
        network_input[i] = notes[i:i + sequence_length,:]
        network_output[i] = notes[i + sequence_length]

    n_patterns = len(network_input)
#     print(network_input)
#     print(network_output)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 88))
    # normalize input
#     network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, note_num):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(note_num))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=5, batch_size=64, callbacks=callbacks_list)


if __name__ == '__main__':
    pass
    #train_network()


In [4]:
print(get_notes())
print(prepare_sequences())

Parsing test/BlueStone_LastDungeon.mid
Parsing test/8.mid
(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 382)
